In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tqdm
import io
import imageio

from collections import deque
from skimage.transform import resize, rescale
from skimage.measure import compare_ssim as ssim
from scipy.cluster.vq import vq

import matplotlib.pyplot as plt

# Own libraries and modules
from compression import jpeg_helpers
from helpers import loading, plotting, utils, summaries, tf_helpers, dataset
from models import compression
from pyfse import pyfse

from training.compression import train_dcn, visualize_distribution, visualize_codebook

In [ ]:
training = {
    'n_epochs': 1501,
    'batch_size': 40,
    'patch_size': 128,
    'sample_dropout': False,
    'learning_rate': 1e-4,
    'learning_rate_reduction_schedule': 1000,
    'learning_rate_reduction_factor': 0.5,    
    'validation_schedule': 100,
    'convergence_threshold': 1e-4,
    'current_epoch': 0,
    'validation_is_training': True,
    'augmentation_probs': {
        'resize': 0.0,
        'flip_h': 0.5,
        'flip_v': 0.5
    }
}

In [ ]:
patch_size = 128
n_latent = 1024 * 8
n_latent_bytes = 0.1

bitmap_size = patch_size * patch_size * 3
compression_rate = bitmap_size / (n_latent_bytes * n_latent)
compression_bpp = 8 * n_latent * n_latent_bytes / patch_size / patch_size

print('Bitmap size: {:,d} bytes'.format(bitmap_size))
print('Latent size: {:,}-D'.format(n_latent))
print('Latent repr: {:,} bytes'.format(n_latent * n_latent_bytes))
print('Compression rate: 1:{}'.format(compression_rate))
print('Compression Fi  : {:.2f} bpp'.format(compression_bpp))

## Load the dataset

In [ ]:
# Load data
data_directory = os.path.join('./data/raw/nip_training_data/Nikon D90/')

data = dataset.IPDataset(data_directory, n_images=5, v_images=5, load='xy', val_rgb_patch_size=128)

print('Sampling ref size: ', data.W, 'x', data.H)
for key in ['training', 'validation']:
    if 'x' in data[key]: print('{}/X : {}'.format(key, data[key]['x'].shape))
    if 'y' in data[key]: print('{}/Y : {}'.format(key, data[key]['y'].shape))
    if key == 'training':
        batch_x, batch_y = data.next_training_batch(0, 5, rgb_patch_size=128)
        print('training/batch', 'x', batch_x.shape if batch_x is not None else None, 'y', batch_y.shape if batch_y is not None else None)
    if key == 'validation':        
        batch_x, batch_y = data.next_validation_batch(0, 5)
        print('validation/batch', 'x', batch_x.shape if batch_x is not None else None, 'y', batch_y.shape if batch_y is not None else None)

In [ ]:
# Load data
data_directory = os.path.join('./data/compression/')

# data = dataset.IPDataset(data_directory, n_images=16000, v_images=800, load='y', val_rgb_patch_size=training['patch_size'])
data = dataset.IPDataset(data_directory, n_images=4000, v_images=200, load='y', val_rgb_patch_size=training['patch_size'])
# data = dataset.IPDataset(data_directory, n_images=1000, v_images=50, load='y', val_rgb_patch_size=training['patch_size'])

for key in ['training', 'validation']:
    print('{} : {}'.format(key, data[key]['y'].shape))

In [ ]:
# Show sample batch from the database
if 'batch_id' not in globals():
    batch_id = 0
    n_batches = data['training']['y'].shape[0] // training['batch_size']
    
batch_id = (batch_id + 1) % n_batches

print('Batch id: {} ({})'.format(batch_id, training['batch_size']))

batch_x = data.next_training_batch(batch_id, training['batch_size'], training['patch_size'])
fig = plotting.imsc(batch_x[:8], ncols=8, figwidth=25)

batch_x = data.next_validation_batch(batch_id, training['batch_size'])
fig = plotting.imsc(batch_x[:8], ncols=8, figwidth=25)

## Define the Deep Compression Network Models

In [ ]:
class ResDCN(compression.DCN):
    
    def construct_model(self):
        
        with tf.name_scope('dcn'):

            activation = tf.nn.ResDCN
            last_activation = tf.nn.sigmoid

            print('Building Deep Compression Network with d-latent={}'.format(n_latent))

            net = self.x
            print('net size: {}'.format(net.shape))
        
            # Convolutions
            n_filters = self.n_filters
            
            net = tf.contrib.layers.conv2d(net, 64, self.kernel, stride=2, activation_fn=activation, scope='dcn{}/conv_{}'.format(self.label, 0))
            
            for r in range(self.n_layers):
                net = tf.contrib.layers.conv2d(net, n_filters, self.kernel, stride=2, activation_fn=activation, scope='dcn{}/conv_{}'.format(self.label, r))
            #     print('net size: {}'.format(net.shape))
#                     net = tf.contrib.layers.max_pool2d(net, 2, scope='dcn{}/pool_{}'.format(self.label, r))
                print('net size: {} // {}'.format(net.shape, net))
                n_filters *= self.n_fscale

            # Flatten and get latent representation
            flat = tf.contrib.layers.flatten(net, scope='dcn{}/flat_{}'.format(self.label, 0))
            print('net size: {}'.format(flat.shape))

            latent = tf.contrib.layers.fully_connected(flat, self.n_latent, activation_fn=activation, scope='dcn{}/dense_{}'.format(self.label, 0))
            print('net size: {}'.format(latent.shape))

            inet = tf.contrib.layers.fully_connected(latent, int(flat.shape[-1]), activation_fn=activation, scope='dcn{}/dense_{}'.format(self.label, 1))
            print('net size: {}'.format(inet.shape))
            inet = tf.reshape(net, tf.shape(net), name='dcn{}/reshape_{}'.format(self.label, 0))
            print('net size: {}'.format(inet.shape))

            # Transposed convolutions
            for r in range(self.n_layers):
                inet = tf.contrib.layers.conv2d_transpose(inet, 3 if r == self.n_layers - 1 else n_filters, self.kernel, stride=2, 
                                                          activation_fn=last_activation if r == self.n_layers - 1 else activation,
                                                          scope='dcn{}/tconv_{}'.format(self.label, r))
                print('net size: {}'.format(inet.shape))
                n_filters = n_filters // self.n_fscale

            y = inet

        with tf.name_scope('dcn{}_optimization'.format(self.label)):
            lr = tf.placeholder(tf.float32, name='dcn_learning_rate')
            loss = tf.nn.l2_loss(self.x - y)
            adam = tf.train.AdamOptimizer(learning_rate=lr)
            opt = adam.minimize(loss, var_list=self.parameters)
            
        return y, lr, loss, adam, opt, latent

## Create DCN instance

In [ ]:
graph = tf.Graph()
sess = tf.Session(graph=graph)

dcn = compression.AutoencoderDCN(sess, graph, patch_size=training['patch_size'], n_latent=0, n_filters=8, n_fscale=2, n_layers=2, r_layers=0, 
            rounding='soft', dropout=False, use_batchnorm=True, train_codebook=False, latent_bpf=5, scale_latent=False, entropy_weight=None)

# dcn = TwitterDCN(sess, graph, patch_size=128)

print(dcn.summary())
print(dcn.model_code)
# print(dcn.count_parameters_breakdown())
print('Compression stats:', dcn.compression_stats(n_latent_bytes=0.5))

# dcn.load_model('./data/raw/compression/aedcn/8x8x192')
# dcn.save_model(os.path.join('./data/raw/compression/', dcn.short_name()), 3767)

In [ ]:
from helpers import tf_helpers
tf_helpers.show_graph(dcn.graph.as_graph_def())

## Training

In [ ]:
# dcn.init()
out = dcn.sess.run(dcn.weights, feed_dict={dcn.x: batch_x})
# fig = plotting.imsc(out[0:32, :])

with dcn.graph.as_default():
    histogram = dcn.sess.run(dcn.histogram, feed_dict={dcn.x: batch_x})
    entropy = dcn.sess.run(- tf.reduce_sum(dcn.histogram * tf.log(dcn.histogram)) / 0.6931, feed_dict={dcn.x: batch_x})

plt.plot(histogram)
print(entropy)

print(- np.sum(histogram * np.log2(histogram)))

In [ ]:
train_dcn({'dcn': dcn}, training, data, './data/raw/compression_playground/')

### Save model

In [ ]:
for var in dcn.parameters:
    if np.any(np.isnan(dcn.sess.run(var))):
        print('!! NaNs found in {}'.format(var.name))
    else:
        print('     all ok in {}'.format(var.name))

In [ ]:
dcn.training_step(batch_x, 1e-4)

In [ ]:
dcn.save_model(os.path.join('./data/raw/compression/', dcn.name), training['current_epoch'])

In [ ]:
dcn.init()

In [ ]:
prebn = dcn.sess.run(dcn.pre_bn, feed_dict={dcn.x: batch_x})
bM = np.mean(prebn, axis=(0,1,2))
bV = np.var(prebn, axis=(0,1,2))
pM = dcn.sess.run(dcn.graph.get_tensor_by_name('autoencoderdcn/encoder/bn_0/moving_mean:0'))
pV = dcn.sess.run(dcn.graph.get_tensor_by_name('autoencoderdcn/encoder/bn_0/moving_variance:0'))
print(prebn.shape)
print(bM.shape)
print(bV.shape)
print(pM.shape)
print(pV.shape)

fig, axes = plotting.sub(2)
axes[0].plot(bM, pM, 'o')
axes[1].plot(bV, pV, 'o')

In [ ]:
n_elements = 8
batch_y_bstats = dcn.process(batch_x[:n_elements], is_training=True)
batch_y_pstats = dcn.process(batch_x[:n_elements], is_training=False)
fig = plotting.imsc(batch_x[:n_elements], ncols=np.max((8, n_elements)), figwidth=25, titles='Input')
fig = plotting.imsc(batch_y_bstats[:n_elements], ncols=np.max((8, n_elements)), figwidth=25, titles='is_training=True (bStats)')
fig = plotting.imsc(batch_y_pstats[:n_elements], ncols=np.max((8, n_elements)), figwidth=25, titles='is_training=False (pStats)')

In [ ]:
import matplotlib.pyplot as plt
from helpers import utils

fig = plt.figure(figsize=(20, 4))
ax = fig.gca()
ax.plot(utils.ma_conv(perf['loss']['training'], n=11))
# ax.plot(np.arange(0, len(loss['training']), sampling_rate), utils.ma_conv(loss['validation'], n=3))
ax.plot(np.arange(0, len(perf['loss']['training']), sampling_rate), perf['loss']['validation'], '-o', alpha=0.3)
ax.plot(perf['loss']['training'], '.', alpha=0.1)
ax.legend(['train', 'valid'], loc='upper right')
# ax.set_yscale('log')

In [ ]:
dcn.load_model(os.path.join('./data/raw/compression/', dcn.short_name()))

In [ ]:
# Show a sample and a reconstruction of the current batch
batch_y = dcn.process(batch_x)
f = plotting.imsc(batch_x[0:8], ncols=8, figwidth=20)
f = plotting.imsc(batch_y[0:8], ncols=8, figwidth=20)

In [ ]:
# Show the codebook

def visualize_codebook(dcn):

    qmin = -2 ** (dcn.latent_bpf - 1) + 1
    qmax = 2 ** (dcn.latent_bpf - 1)

    uniform_cbook = np.arange(qmin, qmax + 1)
    codebook = dcn.sess.run(dcn.codebook).reshape((-1)).tolist()
    print(codebook)

    fig = plt.figure(figsize=(10, 1))

    for x1, x2 in zip(codebook, uniform_cbook):
        fig.gca().plot([x1, x2], [0, 1], 'k:')

    fig.gca().plot(codebook, np.zeros_like(codebook), 'x')
    fig.gca().plot(uniform_cbook, np.ones_like(uniform_cbook), 'ro')
    fig.gca().set_ylim([-1, 2])
    fig.gca().set_yticks([])
    fig.gca().set_xticks(uniform_cbook)
    s = io.BytesIO()
    fig.savefig(s, format='png', bbox_inches='tight')
    plt.close(fig)
    return imageio.imread(s.getvalue())[:,:,:3]

fig = plotting.imsc(visualize_codebook(dcn))

## Explore & Understand the Latent Representation

In [ ]:
fig = plotting.imsc(visualize_codebook(dcn))

In [ ]:
np.histogram(batch_z)

In [ ]:
# Show the distribution of the latent features

graph = tf.Graph()
sess = tf.Session(graph=graph)

dcn = compression.AutoencoderDCN(sess, graph, patch_size=training['patch_size'], n_latent=0, n_filters=8, n_fscale=2, n_layers=2, r_layers=0, 
            rounding='soft', dropout=False, use_batchnorm=True, train_codebook=False, latent_bpf=5, scale_latent=False, entropy_weight=None)

dcn.init()

fig = plotting.imsc(visualize_distribution(dcn, data), figwidth=15)

with dcn.graph.as_default():
    histogram = dcn.sess.run(dcn.histogram, feed_dict={dcn.x: batch_x, dcn.is_training: True})
    entropy = dcn.sess.run(- tf.reduce_sum(dcn.histogram * tf.log(dcn.histogram)) / 0.6931, feed_dict={dcn.x: batch_x, dcn.is_training: True})

plt.plot(histogram)
print('TF et entropy', entropy)

print('NP est entropy', - np.sum(histogram * np.log2(histogram)))

bin_centers = np.arange(-32, 32, 0.1)
bin_boundaries = np.convolve(bin_centers, [0.5, 0.5], mode='valid')
bin_centers = bin_centers[1:-1]

batch_z = dcn.compress(batch_x, is_training=True)
# Compute frequencies using a histogram
# freq = np.histogram(batch_z[:], bins=bin_boundaries, normed=False)[0]
hist = np.histogram(batch_z[:], bins=bin_boundaries, density=True)[0]
hist = hist.clip(1e-9)
probs = hist / np.sum(hist)

entropy_emp = np.sum(- probs * np.log2(probs))
print('Empirical entropy', entropy_emp)

In [ ]:
dcn.init()

In [ ]:
# batch_z = dcn.sess.run(dcn.latent_pre, feed_dict={dcn.x: batch_x, dcn.is_training: True})
# op = dcn.graph.get_operation_by_name('autoencoderdcn/encoder/conv_0/LeakyRelu')
tn = dcn.graph.get_tensor_by_name('autoencoderdcn/encoder/conv_0/weights:0')
print(tn.name)
batch_z = dcn.sess.run(tn, feed_dict={dcn.x: batch_x, dcn.is_training: True})

# batch_z = dcn.compress(batch_x)
print(batch_z.shape)
print(batch_z)

In [ ]:
# Show the distribution of the latent features

sample_batch_size = 50
batch_x = data.next_validation_batch(0, sample_batch_size)

# See latent distribution
batch_z = dcn.compress(batch_x)
# batch_z = dcn.compress_soft(batch_x)
batch_z = batch_z.reshape((sample_batch_size, -1)).T
print(batch_z.shape)

bin_centers = np.arange(-32, 32, 0.1)
bin_boundaries = np.convolve(bin_centers, [0.5, 0.5], mode='valid')
bin_centers = bin_centers[1:-1]

hist = np.histogram(batch_z[:], bins=bin_boundaries, normed=True)[0]

# for bin, value in zip(bin_centers, hist):
#     print('{:.3f}'.format(bin), '->', value)   

fig = plt.figure(figsize=(20, 4))
ax = fig.gca()
ax.bar(bin_centers, hist, width=bin_centers[1] - bin_centers[0], color='r')
ax.set_title('Histogram of quantized coefficients')

In [ ]:
# Show an example of soft quantization & compute entropy

# Prepare the quantization codebook
q_min = np.percentile(batch_z.astype(np.int32), 0.1)
q_max = np.percentile(batch_z.astype(np.int32), 100 - 0.1)

bin_boundaries = np.arange(q_min + 0.5, q_max + 0.5, 1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

print('Bin centers ({}): {}'.format(len(bin_centers), bin_centers.tolist()))

batch_ex = batch_z.T[0]
value = batch_ex.reshape((4096, 1))

# Compute soft quantization
sigma = 0.1
weights = np.exp(-sigma * np.power(value - bin_centers.reshape(1, len(bin_centers)), 2))
weights = weights / weights.sum(axis=1, keepdims=True)
soft = np.sum(weights * bin_centers, axis=1)

# Compute frequencies using unique
indices, _ = vq(batch_z.reshape((-1, )), bin_centers)
unique, counts = np.unique(bin_centers[indices], return_counts=True)
counts = counts / counts.sum()

# Compute soft histogram
histogram = np.mean(weights, axis=0).clip(1e-6)
histogram = histogram / np.sum(histogram)

entropy = - np.sum(counts * np.log2(counts))
entropy_estimate = - np.sum(histogram * np.log2(histogram))

# Print stats
feature_id = 1
print('Feature', feature_id)
print('Value', value[feature_id])
print('Hard Quantization', np.round(value[feature_id]))
print('Soft quantization', np.sum(weights[feature_id] * bin_centers))

# Plot results
fig, axes = plotting.sub(2, figwidth=16, figheight=4)

axes[0].plot(bin_centers, weights[feature_id])
axes[0].set_title('Soft quantization of a selected feature {:.3f} / {:.3f}'.format(batch_ex[feature_id], np.sum(soft[feature_id])))

axes[1].plot(unique, counts)
axes[1].plot(bin_centers, histogram)
axes[1].legend(['unique', 'soft'])
axes[1].set_title('Actual vs. soft estimated distributions - H={:.2f} vs. {:.2f}'.format(entropy_estimate, entropy))

## Entropy Coding

In [ ]:
q_min = np.percentile(batch_z.astype(np.int32), 0.1)
q_max = np.percentile(batch_z.astype(np.int32), 100 - 0.1)

bin_boundaries = np.arange(q_min + 0.5, q_max - 0.5, 1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

# print(bin_boundaries)


batch_z = dcn.compress(batch_x)
# Compute frequencies using a histogram
freq = np.histogram(batch_z[:], bins=bin_boundaries, normed=False)[0]
hist = np.histogram(batch_z[:], bins=bin_boundaries, normed=True)[0]
hist = hist.clip(1e-9)
probs = hist / np.sum(hist)

# Compute frequencies using unique
indices, _ = vq(batch_z.reshape((-1, )), bin_centers)
unique, counts = np.unique(bin_centers[indices], return_counts=True)

print('Bin centers - {} values'.format(len(bin_centers)))
for bin, value in zip(bin_centers, freq):
    print('  ',bin, '->', value.round(2))   

# print(hist.round(2))
# print(probs.round(2))

entropy = np.sum(- probs * np.log2(probs))

print('Naive coding: {:.2f}'.format(np.log2(len(bin_centers))))
print('Entropy: {:.2f}'.format(entropy))

from dahuffman import HuffmanCodec

codec = HuffmanCodec.from_frequencies({k: v for k, v in zip(bin_centers.astype(np.int), freq)})

In [ ]:
{k: v for k, v in zip(bin_centers.astype(np.int), probs)}

In [ ]:
for word codec.get_code_table()

In [ ]:
from scipy.cluster.vq import vq

print(batch_z.T[0].shape)

code_book = bin_centers.astype(np.int)

# Vector quantization
indices, distortion = vq(batch_z.T[0], code_book)
batch_q = code_book[indices]

print('Codebook {}: {}'.format(len(code_book), code_book))
print(batch_z.T[0])
print(batch_q)
print(indices)

encoded = codec.encode(batch_q)
bits_per_symbol = np.ceil(np.log2(len(bin_centers)))

print('Naive coding: {:.0f} bytes'.format(bits_per_symbol * len(batch_q) / 8))
print('Theoretical limit: {:.0f} bytes'.format(entropy * len(batch_q) / 8))
print('Compressed (Huffman): {} bytes'.format(len(encoded)))

In [ ]:
bin_centers

In [ ]:
codec.print_code_table()

In [ ]:
model_output_dirname = os.path.join('./data/raw/compression/', dcn.short_name(), 'raise')
dcn.load_model(model_output_dirname)

In [ ]:
from helpers import plotting

# dcn.init()

batch_x = data.next_validation_batch(0, 256)

# See latent distribution
# batch_z = dcn.compress(batch_x).T
batch_z = dcn.compress(batch_x).reshape((256, -1)).T
# batch_z = batch_x.reshape((256, 128*128*3)).T
print(batch_z.shape)

# cov = batch_z.T * batch_z

# cov = np.cov(batch_z)

cov = np.corrcoef(batch_z)

plt.imshow(cov)

# batch_z = batch_z[:9]

# fig, axes = plotting.sub(len(batch_z) + 1, ncols=10, figwidth=20)

# # print(axes)

# for i, ax in enumerate(axes):
   
#     if i >= len(batch_z):
#         plotting.quickshow(cov)
#     else:        
#         ax.hist(batch_z[i], bins=30)
#         ax.set_yticks([])

In [ ]:
bin_centers = [-3, -2, -1, 0, 1, 2, 3]
p, x = np.histogram(np.random.normal(size=(1000,)), bins=bin_centers)
print(bin_centers)
print(x)
print(np.convolve(x, [0.5, 0.5], mode='valid'))
print(p)
plt.plot(np.convolve(x, [0.5, 0.5], mode='valid'), p)

In [ ]:
latent_means = np.mean(batch_z,axis=1)
fig, axes = plotting.sub(2, figwidth=12)
axes[0].plot(latent_means)
axes[0].set_title('Mean values for all latent variables')
axes[1].hist(latent_means, bins=50, normed=True)
axes[1].set_title('Histogram of mean values for all latent variables')

In [ ]:
batch_x = data.next_validation_batch(0, 400)

batch_z = dcn.compress(batch_x)
batch_z = batch_z.reshape((sample_batch_size, -1)).T
print(batch_z.shape)

n_bins = 64
bin_boundaries = np.linspace(-16, 16, n_bins+1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

print(bin_centers)

distribution = np.zeros((len(batch_z), n_bins))
for i in range(len(batch_z)):
    distribution[i, :] = np.histogram(batch_z[i], bins=bin_boundaries, normed=True)[0]

vis = []
for i in range(len(distribution) // 512):
    vis.append(distribution[i*512:(i+1)*512, :])
    

thumbs = plotting.thumbnails(vis, n_cols=len(vis))
    
fig = plotting.imsc(thumbs, 'A', figwidth=20)

In [ ]:
n_bins = 64
bin_boundaries = np.linspace(-256, 256, n_bins+1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

i = 58
fig = plt.figure(figsize=(16, 4))
fig.gca().fill_between(bin_centers, 0, np.histogram(batch_z[i], bins=bin_boundaries, normed=True)[0], alpha=0.1)
print(distribution[i, :])
print(batch_z[i])

In [ ]:
fig = plt.figure(figsize=(16, 4))
for i in range(len(batch_z)):
#     fig.gca().plot(bin_centers, distribution[i, :])
    fig.gca().fill_between(bin_centers, 0, distribution[i, :], alpha=0.1)

In [ ]:
for op in dcn.sess.graph.get_operations():
    print(op.name)

## Quantization of the latent space

In [ ]:
from scipy.cluster.vq import vq

code_book = np.array([-1, 0, 1])

batch_x = data.next_validation_batch(0, 50)

alpha = 1
n_unique = 64

# See latent distribution
# batch_z = dcn.compress(batch_x).T
batch_z = dcn.compress(batch_x)
# print(batch_z[0, 0:15].round(3))
# batch_z = batch_z.clip(-100, 100)
# batch_z = (alpha * batch_z).astype(np.int32) / alpha
# batch_z = ((alpha * batch_z).astype(np.int32).clip(-n_unique // 2 + 1, n_unique // 2) / alpha)
# print(batch_z[0, 0:15])
print('Int from {} to {}'.format(-n_unique // 2 + 1, n_unique // 2))
print('  unique = {}'.format(np.unique(batch_z)))
print('# unique = {}'.format(len(np.unique(batch_z))))

# Vector quantization
# indices, distortion = vq(batch_z, code_book)
# batch_z = code_book[indices]

batch_y = dcn.decompress(batch_z)

f = plotting.imsc(batch_x[0:8], ncols=8, figwidth=20)
f = plotting.imsc(batch_y[0:8], ncols=8, figwidth=20)

In [ ]:
dim_id = 0
fig = plt.figure(figsize=(8, 8))
fig.gca().hist(batch_z[dim_id])
print(batch_z[dim_id].shape)

In [ ]:
dcn.save_model('./data/raw/compression/aedcn/8x8x192', epoch)

## Experiments with Entropy Coding

In [ ]:
from test_dcn import restore_model
from dahuffman import HuffmanCodec

dirname = './data/raw/compression_scenarios_final/{}/autoencoderdcn/'

dcn_model = 'AutoencoderDCN-8192D/16x16x32-3C2R+BN-r:identity-Q-5.0bpf-S-'

dcn = restore_model(dirname.format(dcn_model), patch_size=256)

print(dcn._h.to_dict())

In [ ]:
## Load an example image

image = imageio.imread('./data/clic256/alberto-montalesi-176097.png').astype(np.float32) / (2**8 - 1)
# image = resize(image, (dcn.patch_size, dcn.patch_size))
batch_x = np.expand_dims(image, axis=0)
# batch_x = utils.slidingwindow(image, 128)
# batch_x = batch_x[6:7]
fig = plotting.imsc(batch_x, figwidth=4)

In [ ]:
counts = np.histogram(batch_z.reshape((-1,1)), bins=code_book_edges, density=True)[0]

print(code_book.tolist())
print(code_book_edges.tolist())
print(counts)

plt.plot(code_book, np.ones_like(code_book), 'x')
plt.plot(code_book_edges[1:-1], np.ones_like(code_book_edges[1:-1]), '|')

plt.plot(code_book, counts, '-')

In [ ]:
density = True

plt.figure(figsize=(20,4))
plt.hist(batch_z.reshape((-1,1)), bins=code_book, density=density, alpha=0.5)
plt.hist(batch_z.reshape((-1,1)), bins=500, density=density, alpha=0.5)
plt.plot(code_book, np.histogram(batch_z.reshape((-1,1)), bins=utils.bin_egdes(code_book), density=density)[0], 'k-')
plt.plot(code_book, utils.qhist(batch_z, code_book, density=density), 'o--')
plt.legend(['hist/codebook', 'hist/uniform', 'np.hist/codebook', 'utils.hist/codebook'])

In [ ]:
## Compare the DCN and JPEG

batch_z = dcn.compress(batch_x, is_training=False)
batch_d = dcn.process(batch_x, is_training=False)

if dcn._h.rounding == 'identity':
    code_book = np.array(code_books[32]).reshape((-1,))
#     code_book = np.linspace(-2, 2, 256)
else:
    code_book = dcn.sess.run(dcn.codebook).reshape((-1,))
    code_book_edges = np.convolve(code_book, [0.5, 0.5], mode='valid')

indices, _ = vq(batch_z.reshape((-1, )), code_book)
print('Codebook size: {} // {}'.format(len(code_book), code_book.tolist()))
counts = utils.qhist(batch_z, code_book)

counts = counts.clip(min=1)
probs = counts / counts.sum()
entropy = - np.sum(probs * np.log2(probs))

# Plot distributions --------------------------------------------------------------------------------------
density = True

plt.figure(figsize=(10,4))
plt.hist(batch_z.reshape((-1,1)), bins=code_book, density=density, alpha=0.5)
plt.hist(batch_z.reshape((-1,1)), bins=500, density=density, alpha=0.5)
plt.plot(code_book, np.histogram(batch_z.reshape((-1,1)), bins=utils.bin_egdes(code_book), density=density)[0], 'k-')
plt.plot(code_book, utils.qhist(batch_z, code_book, density=density), 'o--')
plt.legend(['hist/codebook', 'hist/uniform', 'np.hist/codebook', 'utils.hist/codebook'])
# ---------------------------------------------------------------------------------------------------------

# Construct a Huffman codec
codec = HuffmanCodec.from_frequencies({k: v for k, v in zip(range(len(code_book)), counts)})

# Vector quantization
indices, distortion = vq(batch_z.reshape((-1)), code_book)
batch_q = code_book[indices]
batch_y = dcn.decompress(batch_q.reshape(dcn.latent_shape))

coded_image = codec.encode(indices.tolist())

print(batch_x.min(), batch_x.max())
print(batch_y.min(), batch_y.max())
print(batch_d.min(), batch_d.max())

ssim_value = ssim(batch_x[0], batch_y[0], multichannel=True, data_range=1)
ssim_value_direct = ssim(batch_x[0], batch_d[0], multichannel=True, data_range=1)

# ssim_value = msssim(batch_x[0], batch_y[0])
# ssim_value_direct = msssim(batch_x[0], batch_d[0])

coded_fse = pyfse.easy_compress(bytes(indices.astype(np.uint8)))

print('DCN             : {}'.format(dcn.model_code))
print('Pixels          : {}x{} = {:,}'.format(image.shape[0], image.shape[1], np.prod(image.shape[:2])))
print('Bitmap          : {:,} bytes'.format(np.prod(image.shape)))
print('Batch size      : {:,} elements'.format(np.prod(batch_x.shape)))
print('Code-book size  : {} elements'.format(len(code_book)))
print('Entropy         : {:.2f} bits per symbol'.format(entropy))
print('Latent size     : {:,}'.format(np.prod(batch_z.shape)))
print('PPF Naive       : {:,.0f} --> {:,.0f} bytes [{} bits per element]'.format(
    np.prod(batch_z.shape) * np.log2(len(code_book)) / 8,
    np.prod(batch_z.shape) * np.ceil(np.log2(len(code_book))) / 8,
    np.ceil(np.log2(len(code_book)))
    ))
print('PPF Theoretical : {:,.0f} bytes ({:.2f} bpp)'.format(np.prod(batch_z.shape) * entropy / 8, 
                                                            np.prod(batch_z.shape) * entropy / np.prod(batch_x.shape[1:3])))
print('PPF Huffman     : {:,} bytes ({:.2f} bpp) --> ssim: {:.3f}'.format(len(coded_image), 
                                                              8 * len(coded_image) / np.prod(batch_x.shape[1:3]),
                                                              ssim_value))
print('FSE Coded       : {:,.0f} bytes ({:.2f} bpp)'.format(len(coded_fse), 
                                                            8 * len(coded_fse) / np.prod(batch_x.shape[1:3])))

try:
    jpeg_quality = jpeg_helpers.match_ssim(batch_x[0], ssim_value, subsampling='4:4:4')
except:
    jpeg_quality = 95
    
# jpeg_quality = 75

# Encode JPEG
# imageio.imwrite('test.jpg', batch_x[0], quality=jpeg_quality, subsampling='4:4:4')
batch_j, jpeg_bytes = jpeg_helpers.compress_batch(batch_x, jpeg_quality, effective=True, subsampling='4:4:4')
ssim_value_jpeg = ssim(batch_x[0], batch_j[0], multichannel=True, data_range=1)
# ssim_value_jpeg = msssim(batch_x[0], batch_d[0])
jpeg_bytes = jpeg_bytes[0]

print('JPEG (Q={:2d})     : {:,} bytes ({:0.2f} bpp) --> ssim: {:.3f}'.format(
    jpeg_quality, jpeg_bytes, 8 * jpeg_bytes / np.prod(batch_j.shape[1:3]), ssim_value_jpeg ))

example_id = 0
batch_all = np.concatenate((batch_x[example_id:example_id+1], batch_d[example_id:example_id+1], batch_y[example_id:example_id+1], batch_j[example_id:example_id+1]), axis=0)

plot_titles = ['Original', 'DCN direct ({:.2f})'.format(ssim_value_direct), 'DCN codec ({:.3f})'.format(ssim_value), 'JPEG Q={} ({:.3f})'.format(jpeg_quality, ssim_value_jpeg)]
f = plotting.imsc(batch_all, titles=plot_titles, ncols=len(batch_all)/2, figwidth=15)

In [ ]:
counts = utils.qhist(batch_z, code_books[bins])
cc = plt.hist(batch_z.reshape((-1,1)), bins=50)

cc = np.histogram(batch_z.reshape((-1,1)))[0]

print(sum(cc))
print(sum(counts))

In [ ]:
print(code_book)
np.trapz(np.ones_like(code_book), code_book)

In [ ]:
bins = 32
plt.hist(batch_z.reshape((-1,1)), bins=bins, density=True)
plt.plot(code_books[bins], utils.qhist(batch_z, code_books[bins], density=True))

In [ ]:
code_book = sorted(quantization.generate_codebook(
    np.abs(batch_z.reshape((-1,1))[::10]), 
    size_codebook=bins
)[0])

In [ ]:

np.sort(np.concatenate((-np.array(code_book), np.zeros((1,1)), np.array(code_book)), axis=0).reshape(-1,))

In [ ]:
plt.plot(code_book, np.ones_like(code_book), 'x')

In [ ]:
add_zero = False

code_books = {}
for bins in [4, 8, 16, 32, 64, 128, 256]:
    print(bins)
    
    code_book = sorted(quantization.generate_codebook(
        np.abs(batch_z.reshape((-1,1))[::10]), 
        size_codebook=bins//2
    )[0])
    
    if add_zero:
        code_books[bins] = np.sort(np.concatenate((-np.array(code_book[:-1]), np.zeros((1,1)), np.array(code_book)), axis=0).reshape(-1,))
    else:
        code_books[bins] = np.sort(np.concatenate((-np.array(code_book), np.array(code_book)), axis=0).reshape(-1,))
        
    print(len(code_books[bins]))

In [ ]:
for bins in [4, 8, 16, 32, 64, 128, 256]:
    plt.plot(code_books[bins], 'o-')

In [ ]:
print('{')
for k, v in code_books.items():
    pp = ', '.join(['{:.6f}'.format(x) for x in v])
    print('  {}: np.array([{}]),'.format(k, pp))
print('}')    

In [ ]:
bins = 32
plt.figure(figsize=(20,2))
plt.plot(code_books[bins], np.ones_like(code_books[bins]), 'x')

In [ ]:
plt.plot(
    batch_z.reshape((-1,1)),
    batch_q.reshape((-1,1)),
    '.'
)

In [ ]:
from compression import afi

out_a, out_b = afi.dcn_compare(dcn, batch_x)

diff = np.abs(out_a - out_b)
diff /= diff.max()

fig = plotting.imsc((out_a, out_b, diff), ncols=3, figwidth=30)

In [ ]:
batch_p = data.next_validation_batch(0, 5)

batch_g = utils.batch_gamma(batch_p)

fig = plotting.imsc(batch_p, ncols=5, figwidth=20)
fig = plotting.imsc(batch_g, ncols=5, figwidth=20)

## How to package everything into a file format

In [ ]:
image_struct = {
    'height': batch_x.shape[0],
    'width': batch_y.shape[1],
    'table': codec.get_code_table(),
    'data': coded_image
}

data =pickle.dumps(image_struct)
print(len(data))

In [ ]:
class EntropyCoder(object):
    
    def __init__(self, counts):
        pass
    
    def encode(self, symbols):
        pass
    
    def decode(self, stream):
        pass
    
    def encode_table(self):
        pass
    
    def decode_table(self):
        pass

In [ ]:
import h5py

In [ ]:
import h5py

with h5py.File('data.h5', 'w') as hf:
    hf.create_dataset('code', data=b'cjhasb cgukgdbt67xbr3tr78t32ctb6r2')
    hf.create_dataset('dataset_2', data="{'EOF': 1, '1': 2, '0': 0}")